In [1]:
from models import *
import os
import joblib
import random
import numpy as np
torch.backends.cudnn.enabled=False
from sklearn.metrics import roc_auc_score
import time


torch.backends.cudnn.deterministic = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sparsity = 0.1
bag_size = 10
X_test = joblib.load('../MNIST/X_test_%d_%d_%d'%((int(sparsity*100), bag_size, 250)))
y_test = joblib.load('../MNIST/y_test_%d_%d_%d'%((int(sparsity*100), bag_size, 250)))
actual_y_test = joblib.load('../MNIST/actual_y_test_%d_%d_%d'%((int(sparsity*100), bag_size, 250)))

test_dg = CIFAR10Dataset('test', {'features':torch.tensor(X_test), 'labels':torch.tensor(y_test), 'labels2':torch.tensor(actual_y_test)})
test_loader = DataLoader(test_dg,batch_size = 1,shuffle = False)

### ABDMIL_NOPE

In [13]:
cM = 40
cE = 100
cL = 500

cD = 128
    
model = ABDMIL(ftEx, cM, cE, cL, cD).to(device)
bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)
    
    # positive pair, with encoding
    _, bag_prediction = model(x_i.to(device))

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.24195289611816406


### ABDMIL_PE

In [14]:
cM = 40
cE = 100
cL = 500

cD = 128
cLR = 1e-4
cWD = 1e-7

model = ABDMIL(ftEx, cM, cE, cL, cD, PE = True).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)
    
    # positive pair, with encoding
    _, bag_prediction = model(x_i.to(device))

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.2575087547302246


### CLAM NOPE

In [4]:
cM = 40
cE = 100
cL = 500

cK = 5
cDrop = 0 
cC = 0.7

model = ClamWrapper(ftEx, cM, cE, cL, cK, cDrop, True, PE = False).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction, inst_dict = model(x_i.to(device), y_i.to(torch.int64), instance_eval = True)

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)


0.34418773651123047


### CLAM PE

In [5]:
cM = 40
cE = 100
cL = 500

# K:5, Drop:1, cC:0.5, LR:1e-4, WD:1e-7
cLR = 1e-4
cWD = 1e-7

cK = 5
cDrop = 1
cC = 0.5

model = ClamWrapper(ftEx, cM, cE, cL, cK, cDrop, True, PE = True).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction, inst_dict = model(x_i.to(device), y_i.to(torch.int64), instance_eval = True)

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.3661172389984131


## CLAM MB NOPE

In [6]:
cM = 40
cE = 100
cL = 500

#  K:1, Drop:1, cC:0.5, LR:1e-4, WD:1e-7
cLR = 1e-4
cWD = 1e-7

cK = 1
cDrop = 1
cC = 0.5
        
model = ClamWrapper(ftEx, cM, cE, cL, cK, cDrop, False, PE = False).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction, inst_dict = model(x_i.to(device), y_i.to(torch.int64), instance_eval = True)

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.38845252990722656


## CLAM MB PE

In [7]:
cM = 40
cE = 100
cL = 500

# K:5, Drop:1, cC:0.3, LR:1e-4, WD:1e-6
cLR = 1e-4
cWD = 1e-6

cK = 5
cDrop = 1
cC = 0.3

model = ClamWrapper(ftEx, cM, cE, cL, cK, cDrop, False, PE = True).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction, inst_dict = model(x_i.to(device), y_i.to(torch.int64), instance_eval = True)

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.40518999099731445


## DTFD NOPE

In [8]:
cM = 40
cE = 100
cL = 500

# D:128, LR:1e-4, WD:1e-6, NPB:5
cD = 128
cLR = 1e-4
cWD = 1e-6
num_pseudo_bags = 5

model = DTFD(ftEx, cM, cE, cL, cD, num_pseudo_bags).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
criterion = nn.CrossEntropyLoss()

for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device) 

    bag_prediction, loss_bag = model(x_i.to(device), y_i, criterion)
    
    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())
    del x_i
    del bag_prediction
    del loss_bag
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.6873672008514404


## DTFD PE

In [9]:
cM = 40
cE = 100
cL = 500

#  D:128, LR:1e-4, WD:1e-6, NPB:5
cD = 128
cLR = 1e-4
cWD = 1e-6
num_pseudo_bags = 5

model = DTFD(ftEx, cM, cE, cL, cD, num_pseudo_bags, PE = True).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
criterion = nn.CrossEntropyLoss()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device) 

    bag_prediction, loss_bag = model(x_i.to(device), y_i, criterion)
    
    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())
    del x_i
    del bag_prediction
    del loss_bag
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

0.6968612670898438


### Transformer NOPE

In [10]:
cD = 64
cLR = 1e-4
cWD = 1e-6

model = Transformer(ftEx, cM, cE, cL, cD).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction = model(x_i.to(device))

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)


1.228430986404419


### TRANSFORMER PE

In [11]:
cM = 40
cE = 100
cL = 500

# D:128, LR:1e-4, WD:1e-7
cD = 128
cLR = 1e-4
cWD = 1e-7

model = Transformer(ftEx, cM, cE, cL, cD, PE = True).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction = model(x_i.to(device))

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction

end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)


1.2693581581115723


### TRANSMIL

In [12]:
cM = 40
cE = 100
cL = 500

# D:32, LR:1e-4, WD:1e-6
cD = 32
cLR = 1e-4
cWD = 1e-6

model = TransMIL(ftEx, cM, cE, cL, cD).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)

    # positive pair, with encoding
    bag_prediction = model(x_i.to(device))

    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

1.2892742156982422
